Takes a report sent as an attachment, combines it with another report sent as an attachment, then emails the output to 4 recipients.

In [1]:
import pandas as pd
import win32com.client
import shutil
import re
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders

original = r'C:\Users\USER\Downloads\Product Reserves Report.xlsx'
target = r'C:\\Users\\USER\\OneDrive - COMPANY\\Data Projects\Backorders\Product Reserves Report.xlsx'

shutil.copyfile(original, target)

dfProductReserves = pd.read_excel(target)

dfProductReserves['sku'] = dfProductReserves['sku'].str.replace(' ', '')

#copy over the latest attachment/report from Eagle for the entire inventory Store 7
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# Select main Inbox
inbox = outlook.GetDefaultFolder(6).Folders.Item('Epicor Reports')
messages = inbox.Items

subject = 'Epicor Compass Query Committed Inventory'
path = r'C:\Users\USER\OneDrive - COMPANY\Data Projects\Backorders\\'

try:
    for message in messages:
        try:
            s = message.sender
            s = str(s)
            ss = message.subject
            ss = str(ss)
            # if s == sender & ss == subject:
            if ss == subject:
                attachments = message.Attachments
                # return the first item in attachments
                attachment = attachments.Item(1)
                # the name of attachment file      
                attachment_name = str(attachment).lower()
                attachment.SaveAsFile(path + '\\' + attachment_name)
        except:
            print('Attachment NOT Saved')
            pass
except:
    pass

dfCommittedInventory = pd.read_csv(r'C:\Users\USER\OneDrive - COMPANY\Data Projects\Backorders\committedinventory.csv', converters={'Department': str, 'Item Number': str})

from functools import reduce

# compile the list of dataframes you want to merge
data_frames = [dfCommittedInventory, dfProductReserves]

dfMerged = reduce(lambda left,right: pd.merge(left,right,left_on=['Item Number'], right_on=['sku'], how='left'), data_frames).fillna('')

dfMerged.drop(dfMerged.columns[[16, 17, 18]], axis = 1, inplace = True)

dfMerged.drop_duplicates(subset ="Item Number", keep = False, inplace = True)

dfMerged = dfMerged.rename({'product_quantity': 'QOH_NO', 'warehouse_reserve': 'QRES_NO', 'QAVAIL': 'QAVAIL_Eagle'}, axis = 1)

dfMerged['QAVAIL_Eagle'] = dfMerged['QAVAIL_Eagle'].replace('', 0)

dfMerged['QRES_NO'] = dfMerged['QRES_NO'].replace('', 0)

dfMerged['Order Point C'] = dfMerged['Order Point C'].replace('', 0)

# dfMerged['QOH_NO'] = dfMerged['QOH_NO'].astype('int')

# dfMerged['QRES_NO'] = dfMerged['QRES_NO'].astype('int')

dfMerged['SYS_AVAIL'] = dfMerged['QAVAIL_Eagle'].astype('int') - dfMerged['QRES_NO'].astype('int')

dfMerged['TRANSFER'] = dfMerged['Order Point C'].astype('int') - dfMerged['SYS_AVAIL']

dfMerged = dfMerged.reindex(columns=['Item Number', 'Item Description', 'SYS_AVAIL', 'TRANSFER', 'Secondary Vendor', 'Manufacturer',
                                      'MFG Part #', 'Department', 'Class', 'Fineline', 'QOH', 'QCOM', 'QAVAIL_Eagle', 'Order Point',
                                      'Order Point A', 'Order Point C', 'Store Closeout?', 'QOH_NO', 'QRES_NO'])

with pd.ExcelWriter(r'C:\Users\USER\OneDrive - COMPANY\Data Projects\Backorders\CommittedProduct.xlsx', engine='xlsxwriter') as writer:

    # Convert the dataframe to an XlsxWriter Excel object.
    dfMerged.to_excel(writer, sheet_name='Sheet1', index=False, header=True)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']

    # Add some cell formats.
    format1 = workbook.add_format({'num_format': '@'})

    # Note: It isn't possible to format any cells that already have a format such
    # as the index or headers or any cells that contain dates or datetimes.

    # Set the column width and format.
    worksheet.set_column('A:A', None, format1)
    worksheet.set_column('H:H', None, format1)

# the computer needs to be on the corporate network for this to work
# either directly or on VPN
# if there is an error, this will be the likely cause
    
## FILE TO SEND AND ITS PATH
filename = 'CommittedProduct.xlsx'
SourcePathName  = r'C:\\Users\\USER\\OneDrive - COMPANY\\Data Projects\\Backorders\\' + filename 

msg = MIMEMultipart()
msg['From'] = 'USER@website.com'
recipients = ['USER1@website.com', 'USER2@website.com', 'USER3@website.com', 'USER4@website.com']
msg['To'] = ", ".join(recipients)
msg['Subject'] = 'Committed Inventory Report'
body = 'Attached.'
msg.attach(MIMEText(body, 'plain'))

## ATTACHMENT PART OF THE CODE IS HERE
attachment = open(SourcePathName, 'rb')
part = MIMEBase('application', "octet-stream")
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
msg.attach(part)

server = smtplib.SMTP('website-com.mail.protection.outlook.com')  ### put your relevant SMTP here
server.ehlo()
server.starttls()
server.ehlo()
server.send_message(msg)
server.quit()
    
print('Done.')

Done.


In [24]:
myColumnCount = len(dfMerged.columns)

i = 0

while i < myColumnCount:
    col = dfMerged.columns[i]
    # Get column index position of column
    col_index = list(dfMerged.columns).index(dfMerged.columns[i])
    print(str(col_index) + ' - ' + col)
    i += 1

0 - Item Number
1 - Item Description
2 - Vendor
3 - Secondary Vendor
4 - Manufacturer
5 - MFG Part #
6 - Department
7 - Class
8 - Fineline
9 - QOH
10 - QCOM
11 - QAVAIL
12 - Order Point
13 - Order Point A
14 - Order Point C
15 - Store Closeout?
16 - warehouse_code
17 - location_type
18 - sku
19 - product_quantity
20 - warehouse_reserve
